In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/04/04 18:32:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/04 18:32:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
df_green.createOrReplaceTempView('green')

In [4]:
df_green.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [5]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [6]:
df_green_revenue.head(10)

[Row(hour=datetime.datetime(2020, 1, 28, 19, 0), zone=134, amount=193.61000000000007, number_records=17),
 Row(hour=datetime.datetime(2020, 1, 22, 19, 0), zone=65, amount=657.0300000000001, number_records=41),
 Row(hour=datetime.datetime(2020, 1, 27, 8, 0), zone=17, amount=85.56, number_records=4),
 Row(hour=datetime.datetime(2020, 1, 2, 9, 0), zone=66, amount=229.39999999999998, number_records=12),
 Row(hour=datetime.datetime(2020, 1, 2, 12, 0), zone=89, amount=310.28000000000003, number_records=14),
 Row(hour=datetime.datetime(2020, 1, 7, 12, 0), zone=66, amount=179.5, number_records=9),
 Row(hour=datetime.datetime(2020, 1, 3, 8, 0), zone=223, amount=165.90000000000003, number_records=9),
 Row(hour=datetime.datetime(2020, 1, 17, 10, 0), zone=41, amount=638.2699999999999, number_records=49),
 Row(hour=datetime.datetime(2020, 1, 2, 11, 0), zone=26, amount=198.6, number_records=7),
 Row(hour=datetime.datetime(2020, 1, 12, 20, 0), zone=247, amount=36.900000000000006, number_records=3)]

In [7]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-28 19:00:00| 134|193.61000000000007|            17|
|2020-01-22 19:00:00|  65| 657.0300000000001|            41|
|2020-01-27 08:00:00|  17|             85.56|             4|
|2020-01-02 09:00:00|  66|229.39999999999998|            12|
|2020-01-02 12:00:00|  89|310.28000000000003|            14|
|2020-01-07 12:00:00|  66|             179.5|             9|
|2020-01-03 08:00:00| 223|165.90000000000003|             9|
|2020-01-17 10:00:00|  41| 638.2699999999999|            49|
|2020-01-02 11:00:00|  26|             198.6|             7|
|2020-01-12 20:00:00| 247|36.900000000000006|             3|
|2020-01-12 15:00:00|  10|             75.92|             1|
|2020-01-24 04:00:00| 129|148.89000000000001|            13|
|2020-01-04 20:00:00|  25| 369.5700000000001|            23|
|2020-01-22 10:00:00|  7

In [13]:
df_green_revenue.write.parquet('data/report/revenue/green')

In [14]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [8]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.createOrReplaceTempView('yellow')

In [9]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [18]:
df_yellow_revenue \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [19]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [10]:
df_join = df_green_revenue.join(df_yellow_revenue, on=['hour', 'zone'], how='outer')

In [11]:
df_join.show()

+-------------------+----+------------------+--------------+------------------+--------------+
|               hour|zone|            amount|number_records|            amount|number_records|
+-------------------+----+------------------+--------------+------------------+--------------+
|2020-01-01 00:00:00|   3|              null|          null|              25.0|             1|
|2020-01-01 00:00:00|   4|              null|          null|1004.3000000000002|            57|
|2020-01-01 00:00:00|   7| 769.7299999999996|            45| 455.1700000000001|            38|
|2020-01-01 00:00:00|  12|              null|          null|             107.0|             6|
|2020-01-01 00:00:00|  13|              null|          null|1214.8000000000002|            56|
|2020-01-01 00:00:00|  18|               7.8|             1|               5.8|             1|
|2020-01-01 00:00:00|  29|              61.3|             1|              null|          null|
|2020-01-01 00:00:00|  36|295.34000000000003|     

In [2]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [3]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [4]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [ ]:
df_join.write.parquet('data/report/revenue/total')

In [6]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [7]:
df_join = spark.read.parquet('data/report/revenue/total')

In [8]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [9]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   3|              null|                null|              25.0|                    1|
|2020-01-01 00:00:00|   4|              null|                null|1004.3000000000002|                   57|
|2020-01-01 00:00:00|   7| 769.7299999999996|                  45| 455.1700000000001|                   38|
|2020-01-01 00:00:00|  12|              null|                null|             107.0|                    6|
|2020-01-01 00:00:00|  13|              null|                null|1214.8000000000002|                   56|
|2020-01-01 00:00:00|  18|               7.8|                   1|               5.8|                    1|
|2020-01-01 00:00:00|  29|  

In [10]:
df_zones = spark.read.parquet('zones/')

In [11]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [12]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')

In [13]:
df_result.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|  Borough|                Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|2020-01-01 00:00:00|   3|              null|                null|              25.0|                    1|         3|    Bronx|Allerton/Pelham G...|   Boro Zone|
|2020-01-01 00:00:00|   4|              null|                null|1004.3000000000002|                   57|         4|Manhattan|       Alphabet City| Yellow Zone|
|2020-01-01 00:00:00|   7| 769.7299999999996|                  45| 455.1700000000001|                   38|         7|   Queens|             Astoria|   Boro Zone|
|2020-01-01 00:00:00| 